# Introduction to Machine Learning: The Perceptron

**Course: Neural Networks and Machine Learning**

---

## Table of Contents
1. [Motivation: From Biology to Mathematics](#motivation)
2. [The Perceptron Model](#model)
3. [Geometric Interpretation](#geometry)
4. [The Learning Algorithm](#algorithm)
5. [Implementation from Scratch](#implementation)
6. [Applications and Examples](#examples)
7. [Limitations and Linear Separability](#limitations)
8. [Exercises](#exercises)

---

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Motivation: From Biology to Mathematics <a name="motivation"></a>

### The Biological Neuron

The perceptron is inspired by biological neurons:
- **Dendrites** receive signals from other neurons
- The **cell body** integrates these signals
- If the integrated signal exceeds a threshold, the **axon** fires

### The Mathematical Abstraction

The perceptron (Rosenblatt, 1957) translates this into mathematics:

$$
y = f\left(\sum_{i=1}^{n} w_i x_i + b\right)
$$

where:
- $x_i$ are input features (like signals from dendrites)
- $w_i$ are weights (synaptic strengths)
- $b$ is the bias (threshold)
- $f$ is the activation function (firing rule)


## 2. The Perceptron Model <a name="model"></a>

### Mathematical Definition

A perceptron is defined by:

$$
\hat{y} = \text{sign}(\mathbf{w}^T \mathbf{x} + b)
$$

Or equivalently, using an augmented weight vector:

$$
\hat{y} = \text{sign}(\mathbf{w}^T \mathbf{x})
$$

where $\mathbf{x} = [1, x_1, x_2, \ldots, x_n]^T$ and $\mathbf{w} = [b, w_1, w_2, \ldots, w_n]^T$

The sign function is:

$$
\text{sign}(z) = \begin{cases}
+1 & \text{if } z \geq 0 \\
-1 & \text{if } z < 0
\end{cases}
$$

### Key Properties

1. **Binary Classification**: Output is either +1 or -1
2. **Linear Decision Boundary**: The perceptron creates a hyperplane in feature space
3. **Supervised Learning**: We need labeled training data

In [ ]:
# Visualizing the sign activation function
z = np.linspace(-5, 5, 1000)
sign_output = np.sign(z)

plt.figure(figsize=(8, 4))
plt.plot(z, sign_output, 'b-', linewidth=2, label='sign(z)')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('z', fontsize=12)
plt.ylabel('sign(z)', fontsize=12)
plt.title('Perceptron Activation Function', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.ylim(-1.5, 1.5)
plt.show()

## 3. Geometric Interpretation <a name="geometry"></a>

### Decision Boundary

The perceptron defines a **hyperplane** in the input space:

$$
\mathbf{w}^T \mathbf{x} + b = 0
$$

For 2D input $(x_1, x_2)$, this becomes a line:

$$
w_1 x_1 + w_2 x_2 + b = 0
$$

Or in slope-intercept form:

$$
x_2 = -\frac{w_1}{w_2}x_1 - \frac{b}{w_2}
$$

### Weight Vector Properties

- The weight vector $\mathbf{w}$ is **perpendicular** to the decision boundary
- $\|\mathbf{w}\|$ determines the "confidence" of classification
- Points on one side of the hyperplane are classified as +1, others as -1

In [ ]:
# Visualizing geometric interpretation
def plot_decision_boundary(w, b, xlim=(-5, 5), ylim=(-5, 5)):
    """
    Plot the decision boundary for a 2D perceptron
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Create a mesh
    x1 = np.linspace(xlim[0], xlim[1], 100)
    x2 = np.linspace(ylim[0], ylim[1], 100)
    X1, X2 = np.meshgrid(x1, x2)
    
    # Calculate decision function
    Z = w[0] * X1 + w[1] * X2 + b
    
    # Plot decision regions
    ax.contourf(X1, X2, Z, levels=[-1000, 0, 1000], colors=['lightblue', 'lightcoral'], alpha=0.3)
    
    # Plot decision boundary
    ax.contour(X1, X2, Z, levels=[0], colors='black', linewidths=2)
    
    # Plot weight vector (perpendicular to boundary)
    origin = np.array([0, 0])
    ax.quiver(*origin, w[0], w[1], scale=1, scale_units='xy', angles='xy', 
              color='red', width=0.008, label='Weight vector w')
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xlabel('$x_1$', fontsize=12)
    ax.set_ylabel('$x_2$', fontsize=12)
    ax.set_title(f'Decision Boundary: $w_1={w[0]}, w_2={w[1]}, b={b}$', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.text(xlim[1]-1, ylim[0]+0.5, 'Class -1', fontsize=12, color='blue')
    ax.text(xlim[0]+0.5, ylim[1]-0.5, 'Class +1', fontsize=12, color='red')
    
    return fig, ax

# Example: visualize a perceptron
w = np.array([2, 1])
b = -3
plot_decision_boundary(w, b)
plt.show()

### 📝 Exercise 3.1 (Easy)

**Question**: Given weights $w_1 = 1$, $w_2 = -2$, and bias $b = 4$:

a) Write the equation of the decision boundary

b) Classify the following points: $(0, 0)$, $(5, 1)$, $(1, 3)$

c) Visualize the decision boundary using the function above

In [ ]:
# Your solution here


## 4. The Learning Algorithm <a name="algorithm"></a>

### The Perceptron Learning Rule

The perceptron uses a simple but powerful learning algorithm:

**For each training example** $(\mathbf{x}^{(i)}, y^{(i)})$:

1. **Predict**: $\hat{y}^{(i)} = \text{sign}(\mathbf{w}^T \mathbf{x}^{(i)})$

2. **Update** (if wrong):
   $$
   \mathbf{w} \leftarrow \mathbf{w} + \eta \cdot (y^{(i)} - \hat{y}^{(i)}) \cdot \mathbf{x}^{(i)}
   $$
   
   where $\eta$ is the **learning rate** (typically 0.01 to 1.0)

### Understanding the Update Rule

- **If correct** ($y^{(i)} = \hat{y}^{(i)}$): No update needed
- **If wrong**:
  - When $y^{(i)} = +1$ but $\hat{y}^{(i)} = -1$: Move $\mathbf{w}$ toward $\mathbf{x}^{(i)}$
  - When $y^{(i)} = -1$ but $\hat{y}^{(i)} = +1$: Move $\mathbf{w}$ away from $\mathbf{x}^{(i)}$

### Perceptron Convergence Theorem

**Theorem (Novikoff, 1962)**: If the training data is **linearly separable**, the perceptron algorithm will converge to a solution in a finite number of steps.

$$
\text{Number of mistakes} \leq \left(\frac{R}{\gamma}\right)^2
$$

where:
- $R$ = maximum norm of training examples
- $\gamma$ = margin (minimum distance to decision boundary)

### Algorithm Pseudocode

```
PERCEPTRON-TRAIN(X, y, η, max_epochs):
    Initialize w = 0, b = 0
    
    for epoch = 1 to max_epochs:
        errors = 0
        
        for each (x_i, y_i) in training data:
            # Predict
            ŷ_i = sign(w^T x_i + b)
            
            # Update if wrong
            if ŷ_i ≠ y_i:
                w ← w + η · y_i · x_i
                b ← b + η · y_i
                errors += 1
        
        if errors == 0:
            break  # Converged!
    
    return w, b
```

## 5. Implementation from Scratch <a name="implementation"></a>

Let's implement a perceptron class from scratch:

In [ ]:
class Perceptron:
    """
    Perceptron classifier implementation
    
    Parameters:
    -----------
    learning_rate : float (default=0.01)
        Learning rate (eta)
    max_epochs : int (default=1000)
        Maximum number of passes over the training data
    random_state : int (default=None)
        Random seed for reproducibility
    """
    
    def __init__(self, learning_rate=0.01, max_epochs=1000, random_state=None):
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.random_state = random_state
        self.weights = None
        self.bias = None
        self.errors_per_epoch = []
        
    def fit(self, X, y):
        """
        Fit the perceptron to training data
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,)
            Target values (+1 or -1)
        """
        # Initialize random number generator
        rgen = np.random.RandomState(self.random_state)
        
        # Initialize weights and bias
        self.weights = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self.bias = 0.0
        
        self.errors_per_epoch = []
        
        # Training loop
        for epoch in range(self.max_epochs):
            errors = 0
            
            for xi, yi in zip(X, y):
                # Calculate prediction
                linear_output = np.dot(xi, self.weights) + self.bias
                y_pred = np.sign(linear_output)
                
                # Update if misclassified
                if y_pred != yi:
                    update = self.learning_rate * yi
                    self.weights += update * xi
                    self.bias += update
                    errors += 1
            
            self.errors_per_epoch.append(errors)
            
            # Check for convergence
            if errors == 0:
                print(f"Converged after {epoch + 1} epochs")
                break
        
        return self
    
    def predict(self, X):
        """
        Predict class labels for samples in X
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Samples to predict
            
        Returns:
        --------
        y_pred : array, shape (n_samples,)
            Predicted class labels (+1 or -1)
        """
        linear_output = np.dot(X, self.weights) + self.bias
        return np.sign(linear_output)
    
    def score(self, X, y):
        """
        Calculate accuracy on test data
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
        y : array-like, shape (n_samples,)
            
        Returns:
        --------
        accuracy : float
            Fraction of correctly classified samples
        """
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

print("Perceptron class implemented successfully!")

## 6. Applications and Examples <a name="examples"></a>

### Example 6.1: Logical AND Function

Let's start with a simple example - teaching a perceptron to learn the AND logic gate.

Truth table:
```
x1  x2  | AND
--------+-----
 0   0  |  0
 0   1  |  0
 1   0  |  0
 1   1  |  1
```

In [ ]:
# AND gate dataset (converting 0/1 to -1/+1 for perceptron)
X_and = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])

y_and = np.array([-1, -1, -1, 1])  # -1 for False, +1 for True

# Train perceptron
perceptron_and = Perceptron(learning_rate=0.1, max_epochs=10, random_state=42)
perceptron_and.fit(X_and, y_and)

# Test predictions
print("\nAND Gate Predictions:")
for xi, yi_true in zip(X_and, y_and):
    yi_pred = perceptron_and.predict(xi.reshape(1, -1))[0]
    print(f"Input: {xi} | True: {yi_true:+d} | Predicted: {yi_pred:+d}")

print(f"\nAccuracy: {perceptron_and.score(X_and, y_and):.2%}")

In [ ]:
# Visualize learning progress
plt.figure(figsize=(10, 4))

# Plot 1: Learning curve
plt.subplot(1, 2, 1)
plt.plot(range(1, len(perceptron_and.errors_per_epoch) + 1), 
         perceptron_and.errors_per_epoch, 'o-', linewidth=2, markersize=8)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Number of Errors', fontsize=12)
plt.title('Learning Curve (AND Gate)', fontsize=14)
plt.grid(True, alpha=0.3)

# Plot 2: Decision boundary
plt.subplot(1, 2, 2)
x1_min, x1_max = -0.5, 1.5
x2_min, x2_max = -0.5, 1.5
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 100),
                       np.linspace(x2_min, x2_max, 100))
Z = perceptron_and.predict(np.c_[xx1.ravel(), xx2.ravel()])
Z = Z.reshape(xx1.shape)

plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=ListedColormap(['blue', 'red']))
plt.scatter(X_and[y_and == -1, 0], X_and[y_and == -1, 1], 
            c='blue', marker='o', s=200, edgecolors='k', label='Class -1')
plt.scatter(X_and[y_and == 1, 0], X_and[y_and == 1, 1], 
            c='red', marker='s', s=200, edgecolors='k', label='Class +1')
plt.xlabel('$x_1$', fontsize=12)
plt.ylabel('$x_2$', fontsize=12)
plt.title('Decision Boundary (AND Gate)', fontsize=14)
plt.legend()
plt.xlim(x1_min, x1_max)
plt.ylim(x2_min, x2_max)

plt.tight_layout()
plt.show()

### Example 6.2: Iris Dataset (2 Features)

Let's apply the perceptron to a real dataset - classifying two species of Iris flowers using only two features.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# Load Iris dataset
iris = load_iris()

# Select only two classes (linearly separable) and two features
# Class 0 (Setosa) vs Class 1 (Versicolor)
X_iris = iris.data[:100, [0, 2]]  # sepal length and petal length
y_iris = iris.target[:100]
y_iris = np.where(y_iris == 0, -1, 1)  # Convert to -1/+1

# Standardize features
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)

# Split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_iris_scaled, y_iris, test_size=0.3, random_state=42, stratify=y_iris
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Train perceptron
perceptron_iris = Perceptron(learning_rate=0.1, max_epochs=100, random_state=42)
perceptron_iris.fit(X_train, y_train)

# Evaluate
train_accuracy = perceptron_iris.score(X_train, y_train)
test_accuracy = perceptron_iris.score(X_test, y_test)

print(f"\nTraining Accuracy: {train_accuracy:.2%}")
print(f"Test Accuracy: {test_accuracy:.2%}")

In [ ]:
# Visualize results
def plot_decision_regions(X, y, classifier, resolution=0.02, title=""):
    """
    Plot decision regions for 2D classification
    """
    markers = ('o', 's')
    colors = ('blue', 'red')
    cmap = ListedColormap(colors)
    
    # Plot decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    # Plot samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                   alpha=0.8, c=colors[idx],
                   marker=markers[idx], s=80,
                   label=f'Class {cl}', edgecolors='black')
    
    plt.xlabel('Sepal length (standardized)', fontsize=12)
    plt.ylabel('Petal length (standardized)', fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)

# Plot training and test results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

plt.subplot(1, 2, 1)
plot_decision_regions(X_train, y_train, perceptron_iris, 
                     title='Training Data (Iris)')

plt.subplot(1, 2, 2)
plot_decision_regions(X_test, y_test, perceptron_iris,
                     title='Test Data (Iris)')

plt.tight_layout()
plt.show()

In [ ]:
# Plot learning curve
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(perceptron_iris.errors_per_epoch) + 1),
         perceptron_iris.errors_per_epoch, 'o-', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Number of Misclassifications', fontsize=12)
plt.title('Perceptron Learning Curve (Iris Dataset)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

### 📝 Exercise 6.1 (Easy)

**Question**: Train a perceptron to learn the OR logic gate.

Truth table:
```
x1  x2  | OR
--------+----
 0   0  |  0
 0   1  |  1
 1   0  |  1
 1   1  |  1
```

a) Create the training data

b) Train the perceptron

c) Visualize the decision boundary

d) What are the learned weights and bias?

In [ ]:
# Your solution here


## 7. Limitations and Linear Separability <a name="limitations"></a>

### The XOR Problem

The most famous limitation of the perceptron was discovered by Minsky and Papert (1969): **it cannot learn XOR**.

XOR truth table:
```
x1  x2  | XOR
--------+-----
 0   0  |  0
 0   1  |  1
 1   0  |  1
 1   1  |  0
```

### Why XOR is Impossible for a Single Perceptron

XOR is **not linearly separable**: there is no single straight line that can separate the classes.

**Mathematical Proof:**

For a perceptron to classify XOR correctly, we need:
- $w_1 \cdot 0 + w_2 \cdot 0 + b < 0$ (for class -1)
- $w_1 \cdot 0 + w_2 \cdot 1 + b > 0$ (for class +1)
- $w_1 \cdot 1 + w_2 \cdot 0 + b > 0$ (for class +1)
- $w_1 \cdot 1 + w_2 \cdot 1 + b < 0$ (for class -1)

From equations 2 and 3: $w_2 > -b$ and $w_1 > -b$

Adding these: $w_1 + w_2 > -2b$

But equation 4 requires: $w_1 + w_2 < -b$

This is a **contradiction**! Therefore, no single perceptron can learn XOR.

In [ ]:
# Demonstrate XOR failure
X_xor = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])

y_xor = np.array([-1, 1, 1, -1])  # XOR labels

# Try to train perceptron on XOR
perceptron_xor = Perceptron(learning_rate=0.1, max_epochs=100, random_state=42)
perceptron_xor.fit(X_xor, y_xor)

# Check predictions
print("\nXOR Predictions (will fail):")
for xi, yi_true in zip(X_xor, y_xor):
    yi_pred = perceptron_xor.predict(xi.reshape(1, -1))[0]
    status = "✓" if yi_pred == yi_true else "✗"
    print(f"{status} Input: {xi} | True: {yi_true:+d} | Predicted: {yi_pred:+d}")

print(f"\nAccuracy: {perceptron_xor.score(X_xor, y_xor):.2%}")

In [ ]:
# Visualize XOR problem
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

datasets = [
    (X_and, y_and, "AND (Linearly Separable)"),
    (X_xor, y_xor, "XOR (NOT Linearly Separable)"),
]

for idx, (X, y, title) in enumerate(datasets):
    plt.subplot(1, 3, idx + 1)
    plt.scatter(X[y == -1, 0], X[y == -1, 1], 
                c='blue', marker='o', s=200, edgecolors='k', label='Class -1')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], 
                c='red', marker='s', s=200, edgecolors='k', label='Class +1')
    plt.xlabel('$x_1$', fontsize=12)
    plt.ylabel('$x_2$', fontsize=12)
    plt.title(title, fontsize=12)
    plt.legend()
    plt.xlim(-0.5, 1.5)
    plt.ylim(-0.5, 1.5)
    plt.grid(True, alpha=0.3)

# Learning curve for XOR
plt.subplot(1, 3, 3)
plt.plot(range(1, len(perceptron_xor.errors_per_epoch) + 1),
         perceptron_xor.errors_per_epoch, 'o-', linewidth=2, color='darkred')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Number of Errors', fontsize=12)
plt.title('XOR: Perceptron Cannot Converge', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Definition: Linear Separability

A dataset is **linearly separable** if there exists a hyperplane that perfectly separates the two classes:

$$
\exists \mathbf{w}, b : \text{sign}(\mathbf{w}^T \mathbf{x}^{(i)} + b) = y^{(i)} \quad \forall i
$$

### Solutions to XOR Problem

1. **Feature Engineering**: Add polynomial features (e.g., $x_1 x_2$)
2. **Multi-Layer Networks**: Stack multiple perceptrons (→ Neural Networks!)
3. **Kernel Methods**: Map to higher-dimensional space (→ SVMs)

This limitation led to the development of **multi-layer perceptrons (MLPs)** and modern deep learning!

### 📝 Exercise 7.1 (Medium)

**Feature Engineering for XOR**

We can make XOR linearly separable by adding a polynomial feature!

**Question**: 
a) Add the feature $x_3 = x_1 \cdot x_2$ to the XOR dataset

b) Train a perceptron on this augmented dataset

c) Verify it now works!

d) Visualize the 3D feature space (optional challenge)

In [ ]:
# Your solution here
# Hint: Create X_xor_augmented with shape (4, 3) where the third column is x1 * x2


## 8. Exercises <a name="exercises"></a>

### Easy Exercises

#### 📝 Exercise 8.1: Understanding the Math

Given a perceptron with weights $\mathbf{w} = [2, -1]^T$ and bias $b = 1$:

a) Calculate the output for inputs: $(0, 0)$, $(1, 2)$, $(3, 1)$, $(-1, 4)$

b) Write the equation of the decision boundary in the form $x_2 = mx_1 + c$

c) Draw the decision boundary by hand (or with matplotlib)

d) What is the distance from the origin to the decision boundary?

In [ ]:
# Your solution here


#### 📝 Exercise 8.2: Logical Gates

a) Train a perceptron to learn the NAND gate (NOT AND)

b) Compare the learned weights with the AND gate

c) Can you predict the weights for NAND before training? Verify your prediction.

In [ ]:
# Your solution here


### Medium Exercises

#### 📝 Exercise 8.3: Effect of Learning Rate

Using the Iris dataset from Example 6.2:

a) Train perceptrons with different learning rates: $\eta \in \{0.001, 0.01, 0.1, 1.0\}$

b) Plot the learning curves (errors vs epochs) for all learning rates on the same graph

c) Which learning rate converges fastest? Which is most stable?

d) What happens if $\eta$ is too large (e.g., $\eta = 10$)?

In [ ]:
# Your solution here


#### 📝 Exercise 8.4: Margin Analysis

Create a linearly separable 2D dataset where one class is clustered around $(0, 0)$ and another around $(4, 4)$.

a) Generate 50 samples per class using `np.random.randn`

b) Train a perceptron and visualize the decision boundary

c) Calculate the **margin** (distance from the closest points to the decision boundary)

d) What happens to convergence speed as you increase the margin by moving classes further apart?

In [ ]:
# Your solution here


#### 📝 Exercise 8.5: Pocket Algorithm

The **Pocket Algorithm** (Gallant, 1990) is a variant of the perceptron that works better on non-separable data.

**Algorithm**: Keep track of the best weights seen so far (the ones with fewest errors).

a) Implement the Pocket Algorithm as a new class `PocketPerceptron`

b) Test it on the XOR dataset

c) Compare its performance with the standard perceptron

d) What is the best accuracy it can achieve on XOR?

In [ ]:
# Your solution here
class PocketPerceptron(Perceptron):
    # Modify the fit method to keep track of best weights
    pass


### Hard Exercises

#### 📝 Exercise 8.6: Multi-Class Perceptron

Extend the perceptron to handle **multi-class classification** using the **one-vs-all** strategy.

a) Implement a `MultiClassPerceptron` class that:
   - Trains K binary perceptrons (one per class)
   - Predicts by choosing the class with highest confidence score

b) Test it on the full Iris dataset (3 classes)

c) Visualize decision boundaries for all three classes

d) Calculate the confusion matrix

In [ ]:
# Your solution here
class MultiClassPerceptron:
    def __init__(self, learning_rate=0.01, max_epochs=1000, random_state=None):
        # Your implementation
        pass
    
    def fit(self, X, y):
        # Train one perceptron per class
        pass
    
    def predict(self, X):
        # Return class with highest score
        pass


#### 📝 Exercise 8.7: Perceptron Convergence Theorem

Verify the **Perceptron Convergence Theorem** experimentally.

The theorem states that for linearly separable data:
$$
\text{Number of mistakes} \leq \left(\frac{R}{\gamma}\right)^2
$$

where:
- $R = \max_i \|\mathbf{x}^{(i)}\|$ (maximum norm)
- $\gamma$ is the margin (distance from points to optimal decision boundary)

**Tasks**:

a) Generate several linearly separable datasets with different margins

b) For each dataset:
   - Calculate $R$ and estimate $\gamma$
   - Train a perceptron and count total mistakes (sum of all errors)
   - Verify that mistakes $\leq (R/\gamma)^2$

c) Plot: mistakes vs $(R/\gamma)^2$ for multiple datasets

d) What happens to convergence time as the margin decreases?

In [ ]:
# Your solution here


#### 📝 Exercise 8.8: Voted Perceptron

The **Voted Perceptron** (Freund & Schapire, 1999) is an ensemble method.

**Idea**: Instead of keeping one weight vector, keep all weight vectors seen during training, along with how long each survived. At test time, take a weighted vote.

a) Implement the `VotedPerceptron` class

b) Compare it with standard perceptron on noisy data:
   - Generate linearly separable data
   - Add 10% label noise (flip some labels randomly)
   - Compare accuracies

c) Plot the ensemble of decision boundaries

d) Why does voting help with noisy data?

In [ ]:
# Your solution here


#### 📝 Exercise 8.9: Kernel Perceptron (Challenge)

The **Kernel Perceptron** uses the kernel trick to learn non-linear decision boundaries.

**Key insight**: The weight vector can be written as:
$$
\mathbf{w} = \sum_{i=1}^{n} \alpha_i y^{(i)} \mathbf{x}^{(i)}
$$

So we only need dot products $\mathbf{x}^{(i)} \cdot \mathbf{x}^{(j)}$, which can be replaced by kernels!

a) Implement a `KernelPerceptron` that:
   - Stores $\alpha_i$ values instead of weights
   - Uses a kernel function $k(\mathbf{x}_i, \mathbf{x}_j)$

b) Implement RBF (Gaussian) kernel:
$$
k(\mathbf{x}_i, \mathbf{x}_j) = \exp\left(-\frac{\|\mathbf{x}_i - \mathbf{x}_j\|^2}{2\sigma^2}\right)
$$

c) Test on the XOR dataset (without feature engineering!)

d) Visualize the non-linear decision boundary

**Hint**: Prediction becomes:
$$
\hat{y} = \text{sign}\left(\sum_{i=1}^{n} \alpha_i y^{(i)} k(\mathbf{x}^{(i)}, \mathbf{x})\right)
$$

In [ ]:
# Your solution here (advanced!)


---

## Summary

### What We Learned

1. **Perceptron Model**: $\hat{y} = \text{sign}(\mathbf{w}^T \mathbf{x} + b)$

2. **Learning Rule**: $\mathbf{w} \leftarrow \mathbf{w} + \eta (y - \hat{y}) \mathbf{x}$

3. **Convergence**: Guaranteed for linearly separable data

4. **Limitations**: Cannot learn XOR (not linearly separable)

5. **Solutions**: Feature engineering, multi-layer networks, kernels

### Key Takeaways

- The perceptron is the foundation of neural networks
- Understanding its limitations led to deep learning
- Simple algorithm, powerful geometric interpretation
- Still useful for large-scale online learning

### Next Topics

- Multi-layer Perceptrons (MLPs)
- Backpropagation
- Deep Neural Networks
- Modern architectures (CNNs, RNNs, Transformers)

---

## References

1. Rosenblatt, F. (1957). "The Perceptron: A Perceiving and Recognizing Automaton"
2. Minsky, M., & Papert, S. (1969). "Perceptrons"
3. Novikoff, A. B. (1962). "On convergence proofs on perceptrons"
4. Freund, Y., & Schapire, R. E. (1999). "Large margin classification using the perceptron algorithm"
5. Bishop, C. M. (2006). "Pattern Recognition and Machine Learning"
6. Goodfellow, I., Bengio, Y., & Courville, A. (2016). "Deep Learning"

---